# DistillBERT Experimentation

In this notebook, we explore sentiment analysis on movie reviews using DistilBERT. Although our initial plan was to use the full BERT model, limited resources led us to switch to DistilBERT.

During preprocessing, we observed that some reviews exceeded the 512-token limit. To handle this, we experimented with two approaches: truncation and chunking.

We then compared two training strategies: (1) fine-tuning all pre-trained DistilBERT layers, and (2) freezing DistilBERT and training two additional layers on top.

Conclusion: Truncation outperformed chunking, and fine-tuning the full DistilBERT model yielded better results than training only the added layers.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DistilBertModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
!pip install transformers

In [ ]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cuda


# Data Preperation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

pos_df = pd.read_csv('/content/drive/MyDrive/it1244/cleanedposfull.csv')
neg_df = pd.read_csv('/content/drive/MyDrive/it1244/cleanednegfull.csv')

pos_df['label'] = 1  # Positive = 1
neg_df['label'] = 0  # Negative = 0

pos_df = pos_df[['FileName', 'Cleaned_Content', 'label']]
neg_df = neg_df[['FileName', 'Cleaned_Content', 'label']]

df = pd.concat([pos_df, neg_df], axis=0, ignore_index=True)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print("Combined dataset shape:", df.shape)
print(df.head())


Mounted at /content/drive
Combined dataset shape: (50000, 3)
    FileName                                    Cleaned_Content  label
0  17568.txt  and how they bore you right out of your mind t...      0
1  14894.txt  its not citizen kane but it does deliver cleav...      1
2  23805.txt  if you like othello youll love this flick sinc...      1
3  13159.txt  i watched the this the other night on a local ...      1
4  10128.txt  well i am so glad i watched this on hbo instea...      0


In [ ]:
# Split the dataset into training and validation sets.
# We use an 80-20 split, where 20% of the data is reserved for validation.
# Stratification is applied on the 'label' column to ensure that both sets have a similar class distribution.
# The random_state parameter is set to 42 to ensure reproducibility of the split.

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

print("Training set size:", train_df.shape)
print("Validation set size:", val_df.shape)

print("\nClass distribution in training set:")
print(train_df['label'].value_counts(normalize=True))

print("\nClass distribution in validation set:")
print(val_df['label'].value_counts(normalize=True))


Training set size: (40000, 3)
Validation set size: (10000, 3)

Class distribution in training set:
label
1    0.5
0    0.5
Name: proportion, dtype: float64

Class distribution in validation set:
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [ ]:
# Load the pre-trained DistilBERT tokenizer (uncased version) for tokenizing input text.
# This tokenizer converts raw text into tokens and corresponding IDs, which are used as inputs to the model.
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


# Method 1: Truncation

In [ ]:
# Tokenize the training data:
train_encodings = tokenizer(
    train_df['Cleaned_Content'].tolist(),
    truncation=True,
    padding=True,
    max_length=512
)

# Tokenize the validation data:
val_encodings = tokenizer(
    val_df['Cleaned_Content'].tolist(),
    truncation=True,
    padding=True,
    max_length=512
)


In [ ]:
# Define a custom dataset class for movie reviews that inherits from PyTorch's Dataset
class MovieReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create training and validation datasets using the custom MovieReviewDataset class
train_dataset = MovieReviewDataset(train_encodings, train_df['label'].tolist())
val_dataset   = MovieReviewDataset(val_encodings, val_df['label'].tolist())

In [ ]:
# Load the pre-trained DistilBERT model for sequence classification with two output labels
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

In [ ]:
# Set up training arguments, which specify hyperparameters and configurations for training
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/it1244/results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=False
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to predicted class
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
# Create a Trainer object that handles the training loop and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: qingzhe2 (qingzhe2-national-university-of-singapore-students-union) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.363000,0.253300,0.923700,0.911120,0.939000,0.924850
2,0.117000,0.278667,0.932600,0.924451,0.942200,0.933241


TrainOutput(global_step=10000, training_loss=0.23283363342508673, metrics={'train_runtime': 4635.8288, 'train_samples_per_second': 17.257, 'train_steps_per_second': 2.157, 'total_flos': 1.059739189248e+16, 'train_loss': 0.23283363342508673, 'epoch': 2.0})

In [ ]:
results = trainer.evaluate()
print("Evaluation Results:", results)


Evaluation Results: {'eval_loss': 0.27816954255104065, 'eval_accuracy': 0.9322, 'eval_precision': 0.9217408274785324, 'eval_recall': 0.9446, 'eval_f1': 0.9330304227578032, 'eval_runtime': 147.4179, 'eval_samples_per_second': 67.834, 'eval_steps_per_second': 8.479, 'epoch': 2.0}


# Method 2: Chunking

In [ ]:
def chunk_text(text, tokenizer, max_length=512, overlap=50):

    # Splits the text into multiple overlapping chunks of up to `max_length` tokens.
    # Overlap ensures context continuity between chunks.
    # Returns a list of text segments, each of which is <= max_length tokens.

    tokens = tokenizer.encode(text, add_special_tokens=False)

    chunked_texts = []
    start = 0

    while start < len(tokens):
        chunk = tokens[start:start+max_length]
        chunk_str = tokenizer.decode(chunk, skip_special_tokens=True)
        chunked_texts.append(chunk_str)

        start += (max_length - overlap)

    return chunked_texts


In [ ]:
chunked_contents = []
chunked_labels = []

for content, label in zip(train_df['Cleaned_Content'], train_df['label']):
    # Split the text into multiple 512-token chunks
    segments = chunk_text(content, tokenizer, max_length=512, overlap=50)

    # Each chunk is treated like a separate training example
    for seg in segments:
        chunked_contents.append(seg)
        chunked_labels.append(label)

val_chunked_contents = []
val_chunked_labels = []

for content, label in zip(val_df['Cleaned_Content'], val_df['label']):
    segments = chunk_text(content, tokenizer, max_length=512, overlap=50)

    for seg in segments:
        val_chunked_contents.append(seg)
        val_chunked_labels.append(label)


Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Tokenize training and validation data after chunking
train_encodings = tokenizer(
    chunked_contents,
    truncation=True,
    padding=True,
    max_length=512
)

val_encodings = tokenizer(
    val_chunked_contents,
    truncation=True,
    padding=True,
    max_length=512
)


In [ ]:
# Create training and validation datasets using the custom MovieReviewDataset class after chunking
train_dataset = MovieReviewDataset(train_encodings, chunked_labels)
val_dataset   = MovieReviewDataset(val_encodings, val_chunked_labels)


In [ ]:
# Set up training arguments, and saving the chunked model
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/it1244/results_chunked',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.213700,0.231006,0.917175,0.928303,0.906425,0.917233
2,0.065400,0.317412,0.922744,0.925193,0.921962,0.923575


TrainOutput(global_step=11282, training_loss=0.24502035260545227, metrics={'train_runtime': 4444.3126, 'train_samples_per_second': 20.306, 'train_steps_per_second': 2.539, 'total_flos': 1.1954387924312064e+16, 'train_loss': 0.24502035260545227, 'epoch': 2.0})

# Freezing BERT and training two additional layers on top.

In [ ]:
# Define a custom model that builds on a pre-trained DistilBERT encoder with a two-layer MLP head.
class DistilBertWithTwoLayerHead(nn.Module):
    # Custom model that:
    # 1) Loads a pre-trained DistilBERT encoder and freezes its weights.
    # 2) Adds a two-layer MLP classification head on top of the frozen encoder.
    # 3) Computes the cross-entropy loss when labels are provided.

    def __init__(self, dropout_rate=0.1, num_labels=2):
        super(DistilBertWithTwoLayerHead, self).__init__()

        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')

        for param in self.distilbert.parameters():
            param.requires_grad = False

        hidden_size = self.distilbert.config.hidden_size

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_labels)
        )

        self.loss_fct = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        cls_rep = last_hidden_state[:, 0, :]

        logits = self.classifier(cls_rep)

        loss = None
        if labels is not None:
            loss = self.loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)


In [ ]:
# Instantiate the custom model with a dropout rate of 0.1 and 2 output labels
model = DistilBertWithTwoLayerHead(
    dropout_rate=0.1,
    num_labels=2
)

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/it1244/results_frozen_2layer',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs_frozen_2layer',
    logging_steps=10,
    load_best_model_at_end=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.420800,0.347469,0.852100,0.886498,0.807600,0.845212
2,0.391800,0.336869,0.856300,0.860555,0.850400,0.855447


TrainOutput(global_step=10000, training_loss=0.37996453809738157, metrics={'train_runtime': 1491.2845, 'train_samples_per_second': 53.645, 'train_steps_per_second': 6.706, 'total_flos': 0.0, 'train_loss': 0.37996453809738157, 'epoch': 2.0})